In [1]:
#IMPORT STATEMENTS 
import PyPDF2
import ollama

In [2]:
#FUNSTION TO PRINT THE PROGRAM OUTPUTS IN THE CONSOLE
import time
def printer(v):
    paragraphs = v.split('\n')
    for paragraph in paragraphs:
        words = paragraph.split()
        for word in words:
            print(word, end=' ')
            time.sleep(0.1)
        print()
        time.sleep(0.1)

In [3]:
#FUNCTION DEFINITONS

# Function to extract text from TEXT/PDF
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Function to print the output in a stream
def output(stream):
    response = ""
    for chunk in stream:
        print(chunk['message']['content'], end="")
        response += chunk['message']['content']
    return response

# Function to extract and store the question paper
def processone(q):
    prompt1=f'''Extract the questions  from the provided text and check for grammatical and spelling mistakes. Correct any mistakes found and provide the clean, corrected text without any unnecessary comments or additional notes. Ensure the text flows naturally and is accurate.
             *Part A (10 x 2 = 20 Marks):*
     1. question
     2. question
     3. question
     4. question
     5. question
     6. question
     7. question
     8. question
     9. question
     10. question

     *Part B (5 x 13 = 65 Marks):*
     11a. question
     11b. question
     12a. question
     12b. question
     13a. question
     13b. question
     14a. question
     14b. question
     15a. question
     15b. question

     *Part C (1 x 15 = 15 Marks):*
     16a. question
     16b. question

Here is the text: {q}
'''
    stream = ollama.chat(
        model='mistral',
        messages=[{'role': 'user', 'content': prompt1}],
        stream=True
    )
    result = output(stream)
    return result



# Funtion to extract the irrelevant questions
def check(question,syllabus):
    prompt3 = f"""
You are acting as a syllabus checker. Your task is to carefully review each questions and compare it with the syllabus content. Only questions that do not match the syllabus should be marked as irrelevant. Questions may come from a related subject, but the focus must be on exact syllabus alignment.

Your Task:
1. Compare each question with the syllabus topics, concepts, and learning objectives.
2. If a question does not directly align with the syllabus, mark it as irrelevant, even if it shares some terminology with the subject.
3. List only the irrelevant questions from the extracted list without any explanation or additional notes.

Extracted Questions:
{question}

Syllabus Content:
{syllabus}

Instructions:
- Be strict about comparing each question to the syllabus without assumptions based on related topics.
- Output only the list of irrelevant questions in the simplest form, without comments or explanations along with their corresponding question number.
"""

    # Call Ollama mistral for processing
    stream = ollama.chat(
        model='mistral',
        messages=[{'role': 'user', 'content': prompt3}],
        stream=True
    )

    # Get the result from the stream
    result = output(stream)
    return result

# Function to generate relevant question to replace the irrelevant ones
def generate_replacement_questions(irrelevant,syllabus):
    prompt4 = f"""

You are tasked with rewriting the following irrelevant questions to align them with the syllabus and follow Bloom's Taxonomy. Each rewritten question must retain its original question number for easy replacement.

### Instructions:
1. Rewrite each irrelevant question so it matches the topics and concepts in the syllabus.
2. Maintain the original question numbers exactly as they are.
3. Ensure the questions follow Bloom’s Taxonomy:
   - *Part A*: Lower-level cognitive skills (Remember, Understand).
   - *Part B*: Mid-level cognitive skills (Apply, Analyze).
   - *Part C*: Higher-level cognitive skills (Evaluate, Create).
4. Ensure each question is unique and relevant to the syllabus.
5. *Only provide the rewritten questions with their original question numbers.* No extra commentary is needed.

### Example:
- If question 5 is irrelevant, rewrite it like this:
   5. [New question]

### Irrelevant Questions:
{irrelevant}

### Syllabus Content:
{syllabus}

do not generate extra question. the number of replaced questions must be equal to the number of irrelevant questions
"""
# Use Ollama's model to process the prompt
    stream = ollama.chat(
        model='mistral',  # Using LLaMA 3.1 for accurate response generation
        messages=[{'role': 'user', 'content': prompt4}],
        stream=True
    )

    # Get the result from the stream
    result2 = output(stream)
    return result2

# Funtion to update the question paper with relevant question replacement
def revised(question,irrelevant,replacement):
    prompt5 = f"""
Rewrite the question paper by replacing specific questions with new ones.

Original question paper:
{question}

Replace the following questions:
{irrelevant}

With the following new questions:
{replacement}

Do not modify any questions that are not in the list of questions to replace.
"""

    # Use Ollama's model to process the prompt
    stream = ollama.chat(
        model='mistral',  # Using LLaMA 3.1 for accurate response generation
        messages=[{'role': 'user', 'content': prompt5}],
        stream=True
    )

    # Get the result from the stream
    result2 = output(stream)
    return result2


In [4]:
# TEXT EXTRACTION FROM THE SYLLABUS AND THE MODEL QUESTION PAPER

q = extract_text_from_txt('model23.txt')
s = extract_text_from_txt('ML1301.txt')

In [39]:
# EXTRACT TEXT FROM THE MODEL QUESTION PAPER
question = processone(q)

In [56]:
#QUESTION
printer(question)

Here is the corrected text: 

*Part A (10 x 2 = 20 Marks):* 
1. What is a movie called Titanic? 2 CO1 BL1 
2. What are the steps involved in the machine learning process? 2 CO1 BL1 
3. When should Bies-vriance decomposition be used? 2 CO2 BL2 
4. Differentiate between Discriminative and Generative models. 2 CO2 BL2 
5. In what situations is PCA (Principal Component Analysis) suitable? 2 CO3 BL2 
6. Explain how Linear Discriminant Analysis (LDA) works. 2 CO3 BL2 
7. What are Transition Probabilities? 2 CO4 BL1 
8. Define the HMM Viterbi algorithm. 2 CO4 BL1 
9. Explain what a Perceptron is. 2 CO5 BL1 
10. Explain what a Perceptron is. 2 CO5 BL1 

*Part B (5 x 13 = 65 Marks):* 
11a. Write the FIND-S algorithm and provide an example to trace it. (13) CO1 BL1, BL2 
OR 
11b. Provide an example to demonstrate the application of K-NN Algorithm 
and predict the sport that Angelina plays. 

12a. Given the attributes of life and competition, classify the activities (13) CO2 BL3, BL4 
when you ar

In [41]:
# EXTRACT THE CONTENT OF THE SYLLABUS
# Function to understand the syllabus
def understand(s):
    prompt2=f'''summarise , what the syllabus {s} is about short and concise withing 5 or 10 sentences'''
    # Call Ollama mistral for processing
    stream = ollama.chat(
        model='mistral',
        messages=[{'role': 'user', 'content': prompt2}],
        stream=True
    )

    # Get the result from the stream
    result = output(stream)
    return result
syllabus = understand(s)

In [57]:
#SYLLABUS
printer(syllabus)

This course, ML1301 - DATA FOUNDATION, spans 45 periods. Its main objective is to acquire knowledge on Data Science and its Foundations. The curriculum covers topics like data process, organization, analysis, visualization, ethics, and recent trends in the field. 

Key areas of study include understanding different types of data (quantitative, qualitative), big data, metadata, distributed file systems, EDA philosophy, data structures, organizational models, extraction and ETL processes, spreadsheet manipulations, Tableau visualizations, ethics, and privacy issues in data science. 

Students will learn to explore the fundamental concepts of Data Science, understand the Data Science process and tools of EDA, address how Organizational structure influences efficiency and effectiveness, analyze and validate data using Spreadsheets and Tableau, and think through the ethics related to data sharing and decision-making. The course aims at building interactive dashboards for business purposes a

In [46]:
# CHECK THE MODEL PAPER FOR IRRELEVANT QUESTIONS
irrelevant = check(question, syllabus)

In [58]:
#IRRELEVANT
printer(irrelevant)

1, 2, 4, 9, 10 (Part A) 
11b, 12a (Part B) 
14b, 15b (Part C) 


In [49]:
# GENERATE A SET OF REPLACEMENT QUESTIONS
replacement = generate_replacement_questions(irrelevant,syllabus)

In [59]:
#REPLACEMENT
printer(replacement)

1. What is the primary purpose of data normalization in a database ? (Part A - Understand) 

2. Define the term 'quantitative data' within the context of Data Science. (Part A - Remember) 

4. Explain how big data differs from traditional data sets in terms of volume, variety, and velocity. (Part A - Understand) 

9. Apply the concepts learned about distributed file systems to design an efficient storage solution for a large dataset. (Part B - Apply) 

10. Analyze a given dataset using descriptive statistics to identify any outliers or trends, and interpret their implications. (Part B - Analyze) 

11b. Given a real-world scenario, suggest an appropriate data structure for storing and organizing the data based on its properties. (Part C - Evaluate) 

12a. Develop a hypothesis about the relationship between two variables in a given dataset, and design an experiment to test this hypothesis using EDA techniques. (Part C - Create) 

14b. Discuss the ethical implications of sharing sensitive

In [51]:
# UPDATE THE QUESTION PAPER WITH THE RELEVANT QUESTIONS
def revised(question, irrelevant, replacement):
    prompt5 = f"""
Rewrite the question paper by substituting specific questions with new ones.

Provide the original question paper content: {question}
Specify the questions that need to be replaced: {irrelevant}
Provide the new questions to replace them with: {replacement}

Only modify the questions listed for replacement, leaving the rest of the question paper unchanged.
MODIFY ALL THE QUESTIONS LISTED FOR REPLACEMENT WITHOUT FAIL!!
"""

    # Use Ollama's model to process the prompt
    stream = ollama.chat(
        model='mistral',  # Using LLaMA 3.1 for accurate response generation
        messages=[{'role': 'user', 'content': prompt5}],
        stream=True
    )

    # Get the result from the stream
    result2 = output(stream)
    return result2
rev = revised(question,irrelevant,replacement)

In [60]:
#FINAL
printer(rev)

*Part A (10 x 2 = 20 Marks):* 
1. [What is the primary purpose of data normalization in a database ?] 2 CO1 BL1 
2. [Define the term 'quantitative data' within the context of Data Science.] 2 CO1 BL1 
3. When should Bies-vriance decomposition be used? 2 CO2 BL2 
4. [Explain how big data differs from traditional data sets in terms of volume, variety, and velocity.] 2 CO2 BL2 
5. In what situations is PCA (Principal Component Analysis) suitable? 2 CO3 BL2 
6. Explain how Linear Discriminant Analysis (LDA) works. 2 CO3 BL2 
7. What are Transition Probabilities? 2 CO4 BL1 
8. Define the HMM Viterbi algorithm. 2 CO4 BL1 
9. [Apply the concepts learned about distributed file systems to design an efficient storage solution for a large dataset.] 2 CO5 BL1 
10. [Analyze a given dataset using descriptive statistics to identify any outliers or trends, and interpret their implications.] 2 CO5 BL1 

*Part B (5 x 13 = 65 Marks):* 
11a. Write the FIND-S algorithm and provide an example to trace it. (

In [ ]:
with open('mm.txt', 'w') as file:
    file.write(rev)